# AMR-Guard
### Infection Lifecycle Orchestrator — Kaggle Demo

| Agent | Role | Model |
|---|---|---|
| 1 · Intake Historian | Patient data, CrCl, MDR risk | MedGemma 4B IT |
| 2 · Vision Specialist | Lab report → structured JSON | MedGemma 4B IT |
| 3 · Trend Analyst | MIC creep, resistance velocity | MedGemma 27B Text IT ¹ |
| 4 · Clinical Pharmacologist | Final Rx + safety check | MedGemma 4B IT + TxGemma 9B ¹ |

> ¹ Substituted with smaller variants on Kaggle T4 (16 GB GPU) — see Section 3.

**Before running this notebook:**
1. Click **Add data** (top-right) → search for **`mghobashy/drug-drug-interactions`** → add it
2. Add your HuggingFace token under **Add-ons → Secrets** as `HF_TOKEN`
3. Accept model licences on HuggingFace (see Section 2)

**Steps:** Clone → Install → Authenticate → Download models → Init KB → Launch app

## 1 · Environment

In [ ]:
import subprocess, torch

gpu_info = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
    capture_output=True, text=True
).stdout.strip()
print(f"GPU  : {gpu_info}")
print(f"Torch: {torch.__version__} · CUDA {torch.cuda.is_available()}")

In [ ]:
import os

GITHUB_REPO = "https://github.com/benghita/AMR-Guard.git"
os.environ["GITHUB_REPO"] = GITHUB_REPO

In [ ]:
%%bash
if [ ! -d /kaggle/working/AMR-Guard ]; then
    git clone "$GITHUB_REPO" /kaggle/working/AMR-Guard
else
    echo "Repo already present — pulling latest"
    git -C /kaggle/working/AMR-Guard pull
fi

In [ ]:
%%capture
!pip install -q \
    "langgraph>=0.0.15" "langchain>=0.3.0" langchain-text-splitters langchain-community \
    "chromadb>=0.4.0" sentence-transformers \
    "transformers>=4.50.0" accelerate bitsandbytes \
    gradio huggingface_hub \
    "pydantic>=2.0" python-dotenv openpyxl pypdf "pandas>=2.0" jq

## 2 · Hugging Face Authentication

Add your token under **Kaggle → Add-ons → Secrets** as `HF_TOKEN`.

Accept the model licences **before** running this notebook:
- MedGemma 4B IT → https://huggingface.co/google/medgemma-4b-it
- TxGemma 2B → https://huggingface.co/google/txgemma-2b-predict

In [ ]:
import os
from huggingface_hub import login

try:
    from kaggle_secrets import UserSecretsClient
    hf_token = UserSecretsClient().get_secret("HF_TOKEN")
    print("Token loaded from Kaggle secrets")
except Exception:
    hf_token = os.getenv("HF_TOKEN", "")
    print("Token loaded from environment" if hf_token else "WARNING: No HF_TOKEN found")

if hf_token:
    login(token=hf_token, add_to_git_credential=False)

## 3 · Download Models

| Model | Agent | VRAM (4-bit) | Kaggle T4 |
|---|---|---|---|
| `google/medgemma-4b-it` | 1, 2, 4 primary | ~3 GB | ✓ |
| `google/medgemma-27b-text-it` | 3 (Trend Analyst) | ~14 GB | marginal — using 4B sub |
| `google/txgemma-9b-predict` | 4 safety check | ~5 GB | ✓ (stacked with 4B: ~8 GB) |
| `google/txgemma-2b-predict` | 4 safety fallback | ~1.5 GB | ✓ |

**Kaggle strategy:** download `medgemma-4b-it` and `txgemma-2b-predict`.  
Swap to the full 27B / 9B on a machine with ≥ 24 GB VRAM by editing the variables below.

In [ ]:
from huggingface_hub import snapshot_download

MEDGEMMA_4B = "google/medgemma-4b-it"      # Agents 1, 2, 4 + Agent 3 sub
TXGEMMA_2B  = "google/txgemma-2b-predict"  # Agent 4 safety sub

# Full models for high-VRAM machines (uncomment to use)
# MEDGEMMA_27B = "google/medgemma-27b-text-it"  # Agent 3 — needs ~14 GB in 4-bit
# TXGEMMA_9B   = "google/txgemma-9b-predict"    # Agent 4 safety — needs ~5 GB in 4-bit

for repo_id in [MEDGEMMA_4B, TXGEMMA_2B]:
    print(f"Downloading {repo_id} …")
    snapshot_download(repo_id=repo_id, ignore_patterns=["*.gguf", "*.ot"])
    print(f"  done")

print("\nAll models downloaded.")

In [ ]:
# Embedding model for RAG (CPU-only, no licence needed)
from sentence_transformers import SentenceTransformer
SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Embedding model ready")

## 4 · Configure & Initialise

In [ ]:
import os
from pathlib import Path

# Drug-drug interactions dataset (mghobashy/drug-drug-interactions)
# Kaggle mounts attached datasets at /kaggle/input/<dataset-name>/
KAGGLE_DDI_DIR = Path("/kaggle/input/drug-drug-interactions")
TARGET_CSV = Path("/kaggle/working/AMR-Guard/docs/drug_safety/db_drug_interactions.csv")

if KAGGLE_DDI_DIR.exists():
    csv_files = list(KAGGLE_DDI_DIR.glob("*.csv"))
    if csv_files:
        src = csv_files[0]
        TARGET_CSV.parent.mkdir(parents=True, exist_ok=True)
        import shutil
        shutil.copy(src, TARGET_CSV)
        size_mb = TARGET_CSV.stat().st_size / 1e6
        print(f"DDI CSV copied: {src.name}  ({size_mb:.1f} MB)")
    else:
        print("WARNING: No CSV found in the dataset mount — did you attach the dataset?")
else:
    print("WARNING: /kaggle/input/drug-drug-interactions not found.")
    print("  → Click 'Add data' and attach: mghobashy/drug-drug-interactions")
    print("  Drug interaction checks will be skipped during this run.")

In [ ]:
# Write .env
env = f"""
MEDIC_ENV=kaggle
MEDIC_QUANTIZATION=4bit

# Agent 1, 2, 4 — MedGemma 4B IT
MEDIC_LOCAL_MEDGEMMA_4B_MODEL={MEDGEMMA_4B}

# Agent 3 — MedGemma 27B Text IT  (subbed with 4B for Kaggle T4)
# To use full 27B: set to google/medgemma-27b-text-it
MEDIC_LOCAL_MEDGEMMA_27B_MODEL={MEDGEMMA_4B}

# Agent 4 safety — TxGemma 9B  (subbed with 2B for Kaggle T4)
# To use full 9B: set to google/txgemma-9b-predict
MEDIC_LOCAL_TXGEMMA_9B_MODEL={TXGEMMA_2B}
MEDIC_LOCAL_TXGEMMA_2B_MODEL={TXGEMMA_2B}

MEDIC_EMBEDDING_MODEL=sentence-transformers/all-MiniLM-L6-v2
MEDIC_DATA_DIR=/kaggle/working/AMR-Guard/data
MEDIC_CHROMA_DB_DIR=/kaggle/working/AMR-Guard/data/chroma_db
""".strip()

with open("/kaggle/working/AMR-Guard/.env", "w") as f:
    f.write(env)
print(".env written")

In [ ]:
import sys
sys.path.insert(0, "/kaggle/working/AMR-Guard")

# Populate SQLite + ChromaDB knowledge base
!python /kaggle/working/AMR-Guard/setup_demo.py

## 5 · Launch the Gradio App

Two tabbed scenarios are exposed in a single Gradio interface:

| Tab | Scenario | Agents active |
|---|---|---|
| **Stage 1 — Empirical** | No lab results yet | Agent 1 (MedGemma 4B) → Agent 4 (MedGemma 4B + TxGemma 2B) |
| **Stage 2 — Targeted** | Culture & sensitivity available | Agent 1 (MedGemma 4B) → Agent 2 (MedGemma 4B) → Agent 3 (MedGemma 4B→27B sub) → Agent 4 (MedGemma 4B + TxGemma 2B) |

`demo.launch(share=True)` prints a public Gradio URL — no extra tunnel needed.

In [ ]:
import json
import sys

sys.path.insert(0, "/kaggle/working/AMR-Guard")

# ── Demo fallback (used when pipeline errors out before models are warm) ──────

def _demo_result(patient_data: dict, labs_text) -> dict:
    result = {
        "stage": "targeted" if labs_text else "empirical",
        "creatinine_clearance_ml_min": 58.3,
        "intake_notes": json.dumps({
            "patient_summary": (
                f"{patient_data.get('age_years')}-year-old {patient_data.get('sex')} · "
                f"{patient_data.get('suspected_source', 'infection')}"
            ),
            "creatinine_clearance_ml_min": 58.3,
            "renal_dose_adjustment_needed": True,
            "identified_risk_factors": patient_data.get("comorbidities", []),
            "infection_severity": "moderate",
            "recommended_stage": "targeted" if labs_text else "empirical",
        }),
        "recommendation": {
            "primary_antibiotic": "Ciprofloxacin",
            "dose": "500 mg",
            "route": "Oral",
            "frequency": "Every 12 hours",
            "duration": "7 days",
            "backup_antibiotic": "Nitrofurantoin 100 mg MR BD × 5 days",
            "rationale": (
                "Community-acquired UTI with moderate renal impairment (CrCl 58 mL/min). "
                "Ciprofloxacin provides broad Gram-negative coverage. No dose adjustment "
                "required above CrCl 30 mL/min."
            ),
            "references": ["IDSA UTI Guidelines 2024", "EUCAST Breakpoint Tables v16.0"],
        },
        "safety_warnings": [],
        "errors": [],
    }
    if labs_text:
        result["vision_notes"] = json.dumps({
            "specimen_type": "urine",
            "identified_organisms": [{"organism_name": "Escherichia coli", "significance": "pathogen"}],
            "susceptibility_results": [
                {"organism": "E. coli", "antibiotic": "Ciprofloxacin", "mic_value": 0.25, "interpretation": "S"},
                {"organism": "E. coli", "antibiotic": "Nitrofurantoin", "mic_value": 16,   "interpretation": "S"},
                {"organism": "E. coli", "antibiotic": "Ampicillin",     "mic_value": ">32", "interpretation": "R"},
            ],
            "extraction_confidence": 0.95,
        })
        result["trend_notes"] = json.dumps([{
            "organism": "E. coli",
            "antibiotic": "Ciprofloxacin",
            "risk_level": "LOW",
            "recommendation": "No MIC creep detected — continue current therapy.",
        }])
    return result


# ── Output formatters ─────────────────────────────────────────────────────────

def _parse_json_field(raw):
    if not raw or raw in ("No lab data provided", "No MIC data available for trend analysis", ""):
        return None
    if isinstance(raw, (dict, list)):
        return raw
    try:
        return json.loads(raw)
    except Exception:
        return None


def format_recommendation(result: dict) -> str:
    lines = ["## ℞ Recommendation\n"]
    rec = result.get("recommendation", {})
    if rec:
        drug  = rec.get("primary_antibiotic", "—")
        dose  = rec.get("dose", "—")
        route = rec.get("route", "—")
        freq  = rec.get("frequency", "—")
        dur   = rec.get("duration", "—")
        lines.append(f"**Drug:** {drug}")
        lines.append(
            f"**Dose:** {dose} &nbsp;·&nbsp; **Route:** {route} "
            f"&nbsp;·&nbsp; **Frequency:** {freq} &nbsp;·&nbsp; **Duration:** {dur}"
        )
        if rec.get("backup_antibiotic"):
            lines.append(f"**Alternative:** {rec['backup_antibiotic']}")
        if rec.get("rationale"):
            lines.append(f"\n**Clinical rationale:** {rec['rationale']}")
        if rec.get("references"):
            lines.append("\n**References:**")
            for ref in rec["references"]:
                lines.append(f"- {ref}")

    intake = _parse_json_field(result.get("intake_notes", ""))
    if isinstance(intake, dict):
        lines.append("\n---\n## Patient Summary")
        if intake.get("patient_summary"):
            lines.append(f"> {intake['patient_summary']}")
        crcl = result.get("creatinine_clearance_ml_min") or intake.get("creatinine_clearance_ml_min")
        if crcl:
            lines.append(f"**CrCl:** {float(crcl):.1f} mL/min")
        if intake.get("renal_dose_adjustment_needed"):
            lines.append("⚠ **Renal dose adjustment required**")
        factors = intake.get("identified_risk_factors", [])
        if factors:
            lines.append(f"**Risk factors:** {', '.join(factors)}")

    warnings = result.get("safety_warnings", [])
    if warnings:
        lines.append("\n---\n## ⚠ Safety Warnings")
        for w in warnings:
            lines.append(f"- {w}")

    errors = result.get("errors", [])
    if errors:
        lines.append("\n---\n## Errors")
        for e in errors:
            lines.append(f"- {e}")

    return "\n".join(lines)


def format_lab_analysis(result: dict) -> str:
    lines = []
    vision = _parse_json_field(result.get("vision_notes", ""))
    trend  = _parse_json_field(result.get("trend_notes", ""))

    if vision is None:
        return "*No lab data processed.*"

    if isinstance(vision, dict):
        lines.append("## Lab Extraction")
        if vision.get("specimen_type"):
            lines.append(f"**Specimen:** {vision['specimen_type'].capitalize()}")
        if vision.get("extraction_confidence") is not None:
            conf = float(vision["extraction_confidence"])
            lines.append(f"**Extraction confidence:** {conf:.0%}")

        orgs = vision.get("identified_organisms", [])
        if orgs:
            lines.append("\n**Identified organisms:**")
            for o in orgs:
                name = o.get("organism_name", "Unknown")
                sig  = o.get("significance", "")
                lines.append(f"- **{name}**" + (f" — {sig}" if sig else ""))

        sus = vision.get("susceptibility_results", [])
        if sus:
            lines.append("\n**Susceptibility results:**")
            lines.append("| Organism | Antibiotic | MIC (mg/L) | Result |")
            lines.append("|---|---|---|---|")
            for s in sus:
                interp = s.get("interpretation", "")
                icon   = {"S": "✓ S", "R": "✗ R", "I": "~ I"}.get(interp.upper(), interp)
                lines.append(
                    f"| {s.get('organism','')} | {s.get('antibiotic','')} "
                    f"| {s.get('mic_value','')} | {icon} |"
                )

    if trend:
        items = trend if isinstance(trend, list) else [trend]
        lines.append("\n## MIC Trend Analysis")
        for item in items:
            if not isinstance(item, dict):
                lines.append(str(item))
                continue
            risk  = item.get("risk_level", "UNKNOWN").upper()
            icon  = {"HIGH": "🚨", "MODERATE": "⚠"}.get(risk, "✓")
            org   = item.get("organism", "")
            ab    = item.get("antibiotic", "")
            label = f"{org} / {ab} — " if (org or ab) else ""
            lines.append(f"**{icon} {label}{risk}** — {item.get('recommendation', '')}")

    return "\n".join(lines) if lines else "*No lab analysis available.*"


# ── Pipeline runner helpers ───────────────────────────────────────────────────

def _build_patient_data(age, weight, height, sex, creatinine,
                        infection_site, suspected_source,
                        medications_str, allergies_str, comorbidities_str):
    return {
        "age_years":               float(age),
        "weight_kg":               float(weight),
        "height_cm":               float(height),
        "sex":                     sex,
        "serum_creatinine_mg_dl":  float(creatinine),
        "infection_site":          infection_site,
        "suspected_source":        suspected_source or f"{infection_site} infection",
        "medications":  [m.strip() for m in medications_str.split("\n") if m.strip()],
        "allergies":    [a.strip() for a in allergies_str.split("\n")    if a.strip()],
        "comorbidities":[c.strip() for c in comorbidities_str.split("\n") if c.strip()],
    }


def run_empirical_scenario(age, weight, height, sex, creatinine,
                           infection_site, suspected_source,
                           medications_str, allergies_str, comorbidities_str):
    """Stage 1 — Empirical: no lab results.
    Active models: MedGemma 4B (Agent 1) → MedGemma 4B + TxGemma 2B (Agent 4).
    """
    patient_data = _build_patient_data(
        age, weight, height, sex, creatinine,
        infection_site, suspected_source,
        medications_str, allergies_str, comorbidities_str,
    )
    try:
        from src.graph import run_pipeline
        result = run_pipeline(patient_data, labs_raw_text=None)
    except Exception as exc:
        result = _demo_result(patient_data, None)
        result["errors"].append(f"[Demo mode — pipeline error: {exc}]")
    return format_recommendation(result)


def run_targeted_scenario(age, weight, height, sex, creatinine,
                          infection_site, suspected_source,
                          medications_str, allergies_str, comorbidities_str,
                          labs_text):
    """Stage 2 — Targeted: lab culture & sensitivity available.
    Active models: MedGemma 4B (Agents 1, 2) → MedGemma 4B→27B sub (Agent 3)
                   → MedGemma 4B + TxGemma 2B (Agent 4).
    """
    patient_data = _build_patient_data(
        age, weight, height, sex, creatinine,
        infection_site, suspected_source,
        medications_str, allergies_str, comorbidities_str,
    )
    labs = labs_text.strip() if labs_text else None
    try:
        from src.graph import run_pipeline
        result = run_pipeline(patient_data, labs_raw_text=labs)
    except Exception as exc:
        result = _demo_result(patient_data, labs)
        result["errors"].append(f"[Demo mode — pipeline error: {exc}]")
    return format_recommendation(result), format_lab_analysis(result)


print("Helper functions loaded.")

In [ ]:
import gradio as gr

INFECTION_SITES = ["urinary", "respiratory", "bloodstream", "skin", "intra-abdominal", "CNS", "other"]


def _patient_inputs():
    """Create patient-demographics input widgets inside the current gr.Blocks context."""
    with gr.Row():
        age            = gr.Number(label="Age (years)",            value=65,   minimum=0,   maximum=120,  precision=0)
        weight         = gr.Number(label="Weight (kg)",            value=70.0, minimum=1,   maximum=300)
        height         = gr.Number(label="Height (cm)",            value=170.0,minimum=50,  maximum=250)
    with gr.Row():
        sex            = gr.Dropdown(label="Biological sex",       choices=["male", "female"], value="male")
        creatinine     = gr.Number(label="Serum Creatinine (mg/dL)", value=1.2, minimum=0.1, maximum=20.0)
        infection_site = gr.Dropdown(label="Infection site",       choices=INFECTION_SITES,    value="urinary")
    suspected_source = gr.Textbox(label="Suspected source",
                                  placeholder="e.g., community-acquired UTI")
    with gr.Row():
        medications    = gr.Textbox(label="Current medications (one per line)",
                                    placeholder="Metformin\nLisinopril", lines=3)
        allergies      = gr.Textbox(label="Drug allergies (one per line)",
                                    placeholder="Penicillin\nSulfa", lines=3)
        comorbidities  = gr.Textbox(label="Comorbidities / MDR risk factors (one per line)",
                                    placeholder="Diabetes\nCKD\nPrior MRSA", lines=3)
    return [age, weight, height, sex, creatinine, infection_site,
            suspected_source, medications, allergies, comorbidities]


with gr.Blocks(title="AMR-Guard · AMR-Guard", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
# ⚕ AMR-Guard — Infection Lifecycle Orchestrator

**Multi-Agent Clinical Decision Support for Antimicrobial Stewardship**

| Model | Agent(s) | Role |
|---|---|---|
| `google/medgemma-4b-it` | 1, 2, 4 | Intake · Lab extraction · Final Rx |
| `google/medgemma-4b-it` (27B sub on T4) | 3 | MIC trend analysis |
| `google/txgemma-2b-predict` (9B sub on T4) | 4 (safety) | Drug interaction screening |

> ⚠ **Research demo only** — not validated for clinical use. All output must be reviewed by a licensed clinician.
---
""")

    with gr.Tabs():

        # ──────────────────────────────────────────────────────────────────────
        # TAB 1 — Stage 1: Empirical (no lab results)
        # ──────────────────────────────────────────────────────────────────────
        with gr.Tab("Stage 1 — Empirical (no lab results)"):
            gr.Markdown("""
**Scenario:** Patient presents without culture / sensitivity data.

**Pipeline:** Agent 1 — *Intake Historian* (MedGemma 4B IT) → Agent 4 — *Clinical Pharmacologist* (MedGemma 4B IT + TxGemma 2B)
""")
            emp_inputs = _patient_inputs()
            emp_btn    = gr.Button("Run Empirical Pipeline", variant="primary")
            emp_output = gr.Markdown(label="Recommendation")

            emp_btn.click(
                fn=run_empirical_scenario,
                inputs=emp_inputs,
                outputs=emp_output,
            )

        # ──────────────────────────────────────────────────────────────────────
        # TAB 2 — Stage 2: Targeted (culture & sensitivity available)
        # ──────────────────────────────────────────────────────────────────────
        with gr.Tab("Stage 2 — Targeted (lab results available)"):
            gr.Markdown("""
**Scenario:** Culture & sensitivity report (any language) is available.

**Pipeline:** Agent 1 (MedGemma 4B IT) → Agent 2 — *Vision Specialist* (MedGemma 4B IT) → Agent 3 — *Trend Analyst* (MedGemma 27B→4B sub) → Agent 4 (MedGemma 4B IT + TxGemma 2B)
""")
            tgt_inputs = _patient_inputs()
            tgt_labs   = gr.Textbox(
                label="Lab / Culture Report — paste text (any language)",
                placeholder=(
                    "Organism: Escherichia coli\n"
                    "Ciprofloxacin: S  MIC 0.25 mg/L\n"
                    "Nitrofurantoin: S  MIC 16 mg/L\n"
                    "Ampicillin: R  MIC >32 mg/L"
                ),
                lines=6,
            )
            tgt_btn = gr.Button("Run Targeted Pipeline", variant="primary")

            with gr.Row():
                tgt_rec_output = gr.Markdown(label="Recommendation")
                tgt_lab_output = gr.Markdown(label="Lab Analysis & MIC Trend")

            tgt_btn.click(
                fn=run_targeted_scenario,
                inputs=tgt_inputs + [tgt_labs],
                outputs=[tgt_rec_output, tgt_lab_output],
            )

    gr.Markdown("""
---
**Knowledge bases:** EUCAST v16.0 · WHO AWaRe 2024 · IDSA AMR Guidance 2024 · ATLAS Surveillance · WHO GLASS · DDInter 2.0  
**Inference:** HuggingFace Transformers · 4-bit quantization · Kaggle T4 GPU
""")

print("Gradio app defined. Run the next cell to launch.")

In [ ]:
# share=True creates a public Gradio URL that anyone can access to interact with the demo.
# The URL is printed below and stays live for ~72 hours.
demo.launch(share=True, quiet=True)